In [31]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
# import matplotlib.backends.backend_qt5agg
import PyQt5
import matplotlib as mpl
# import latex 
mpl.use('Qt5Agg')
# plt.rcParams['text.usetex'] = True

location = r'C:\Users\erikn\OneDrive - Chalmers\Computational Biology\CB HW 2'
task = 3

rho = 0.5
q = 8

L = 100
xi_list = np.linspace(1,L,L)

xi_0 = 20
u0_1 = 5.56155

xi_0 = 50
u0 = 1.43845

xi_0 = 50
u0 = 1.1 * 1.43845

T = 400
dt = 0.1
t_list = np.linspace(0, T, int(T/dt))

def u0_wave_func(u0):

    u_list = np.zeros((len(t_list),len(xi_list)))
    # u = u0

    for i in range(L):

        xi = xi_list[i]
        u = u0 / (1+np.exp(xi-xi_0))
        u_list[0,i] = u

    return u_list

u_list = u0_wave_func(u0)

for t in range(len(t_list)-1):    
    for j in range(len(xi_list)):

        xi = xi_list[j]

        if xi == 1:
            diffusion = (u_list[t, j+1] - u_list[t, 0]) / 1**2
        elif xi == L:
            diffusion = -(u_list[t, j] - u_list[t, j-1]) / 1**2  
        else:
            diffusion = (u_list[t, j+1] + u_list[t, j-1] - 2*u_list[t, j]) / 1**2
        
        u = u_list[t,j]
        u_list[t+1,j] = u + (rho*u*(1-u/q) - u/(1+u) + diffusion)*dt        
       
tau_fixed_early = 2500
tau_fixed_mid = 2750
tau_fixed_late = 3000

fig1, ax = plt.subplots( figsize=(6,6))
ax.plot(xi_list, u_list[0,:], label='$u(\\xi, \\tau=0)$')
ax.plot(xi_list, u_list[tau_fixed_early,:], label='$u(\\xi, \\tau={})$'.format(tau_fixed_early*dt))
ax.plot(xi_list, u_list[tau_fixed_mid,:], label='$u(\\xi, \\tau={})$'.format(tau_fixed_mid*dt))
ax.plot(xi_list, u_list[tau_fixed_late,:], label='$u(\\xi, \\tau={})$'.format(tau_fixed_late*dt))
ax.set_xlabel('$\\xi$', fontsize=13)
ax.set_ylabel('$u(\\xi, \\tau)$', fontsize=13)
ax.set_ylim((0,8))
ax.set_box_aspect(1) 
plt.legend(loc="upper right", prop={'size': 11})

location = r'C:\Users\erikn\OneDrive - Chalmers\Computational Biology\CB HW 2'
title = '/2.1b_wave_{}'.format(task)
plt.savefig(location+title+'.png')


v_list = np.zeros_like(u_list[0,:])

for i in range(len(v_list)-1):
    v = (u_list[tau_fixed_mid,i+1] - u_list[tau_fixed_mid,i]) / 1
    v_list[i] = v

v_last = len(v_list)-1
v_list[v_last] = v_list[v_last-1]

fig2, ax = plt.subplots(figsize=(6,6))
ax.plot(u_list[tau_fixed_mid,:], v_list)
ax.plot(u0_1,0,'.', markersize=15, color='magenta', label='$u_3^*$')
ax.plot(u0,0,'.', markersize=15, color='red', label='$u_2^*$')
ax.plot(0,0,'.', markersize=15, color='green', label='$u_1^*$')
ax.set_xlabel('$u$', fontsize=13)
ax.set_ylabel('$v$', fontsize=13)
plt.legend(loc="lower left", prop={'size': 11})
ax.set_box_aspect(1) 

title = '/2.1b_phase_{}'.format(task)
plt.savefig(location+title+'.png')

plt.show()



In [32]:

import matplotlib.backends
import os.path

def is_backend_module(fname):
    """Identifies if a filename is a matplotlib backend module"""
    return fname.startswith('backend_') and fname.endswith('.py')

def backend_fname_formatter(fname): 
    """Removes the extension of the given filename, then takes away the leading 'backend_'."""
    return os.path.splitext(fname)[0][8:]

# get the directory where the backends live
backends_dir = os.path.dirname(matplotlib.backends.__file__)

# filter all files in that directory to identify all files which provide a backend
backend_fnames = filter(is_backend_module, os.listdir(backends_dir))

backends = [backend_fname_formatter(fname) for fname in backend_fnames]

print(backends)


['agg', 'cairo', 'gtk3', 'gtk3agg', 'gtk3cairo', 'gtk4', 'gtk4agg', 'gtk4cairo', 'macosx', 'mixed', 'nbagg', 'pdf', 'pgf', 'ps', 'qt', 'qt5', 'qt5agg', 'qt5cairo', 'qtagg', 'qtcairo', 'svg', 'template', 'tkagg', 'tkcairo', 'webagg', 'webagg_core', 'wx', 'wxagg', 'wxcairo']


In [33]:

# Numerical solver + streamplot background
umin = -2
vmin = -2
umax = 7
vmax = 2

c = -0.7

# Streamplot
no_points = 100
u_points = np.linspace(umin, umax, no_points)
v_points = np.linspace(vmin, vmax, no_points)
U, V = np.meshgrid(u_points, v_points)

du_dxi_streamplot = V
dv_dxi_streamplot = U/(1+U) - c*V - rho*U*(1-U/q)
'''
# Numerical solver
XI = 20
dxi = 0.1
xi = np.linspace(0, XI, int(XI/dxi)+1)

u0_1 = 0
v0_1 = 0.01

u0_2 = 1.43845 - 0.1
v0_2 = 0

u0_3 = 5.56155 - 0.1
v0_3 = 0
# IC = [u0_solver,v0_solver]
IC = [u0_3,v0_3]


def wave_solver(IC, t):
    u = IC[0]
    v = IC[1]
    du_dxi_solver = v
    dv_dxi_solver = u/(1+u) - c*v - rho*u*(1-u/q)
    return [du_dxi_solver, dv_dxi_solver]

solver = odeint(wave_solver, IC, xi)
u = solver[:,0]
v = solver[:,1]

XI = 100
dxi = 0.1
xi = np.linspace(0, XI, int(XI/dxi)+1)
IC = [u0_1,v0_1]
solver2 = odeint(wave_solver, IC, xi)
u2 = solver[:,0]
v2 = solver[:,1]


# Plotting streamplot and solver
fig3, ax = plt.subplots(figsize=(6,6))
ax.streamplot(U, V, du_dxi_streamplot, dv_dxi_streamplot, density = 2)
# ax.plot(u, v, '-', color='red', linewidth=2)
# ax.plot(u2, v2, '-', color='red', linewidth=2)
# ax.plot(u0_1, v0_1, '.', color='black', markersize=15, label='') # Equilibrium/initial condition
# ax.plot(u0_2, v0_2, '.', color='black', markersize=15, label='') # Equilibrium/initial condition
# ax.plot(u0_3, v0_3, '.', color='black', markersize=15, label='') # Equilibrium/initial condition
ax.set_xlabel('$u$')
ax.set_ylabel('$v$')
# ax.set_xlim(umin,umax)
# ax.set_ylim(umin,umax)
ax.set_box_aspect(1) 

# plt.legend(loc="upper left")
# plt.savefig(location+title+'.png')
plt.show()
'''

C:\Users\erikn\AppData\Local\Temp\ipykernel_13380\1159368571.py:16: RuntimeWarning: divide by zero encountered in divide
  dv_dxi_streamplot = U/(1+U) - c*V - rho*U*(1-U/q)


'\n# Numerical solver\nXI = 20\ndxi = 0.1\nxi = np.linspace(0, XI, int(XI/dxi)+1)\n\nu0_1 = 0\nv0_1 = 0.01\n\nu0_2 = 1.43845 - 0.1\nv0_2 = 0\n\nu0_3 = 5.56155 - 0.1\nv0_3 = 0\n# IC = [u0_solver,v0_solver]\nIC = [u0_3,v0_3]\n\n\ndef wave_solver(IC, t):\n    u = IC[0]\n    v = IC[1]\n    du_dxi_solver = v\n    dv_dxi_solver = u/(1+u) - c*v - rho*u*(1-u/q)\n    return [du_dxi_solver, dv_dxi_solver]\n\nsolver = odeint(wave_solver, IC, xi)\nu = solver[:,0]\nv = solver[:,1]\n\nXI = 100\ndxi = 0.1\nxi = np.linspace(0, XI, int(XI/dxi)+1)\nIC = [u0_1,v0_1]\nsolver2 = odeint(wave_solver, IC, xi)\nu2 = solver[:,0]\nv2 = solver[:,1]\n\n\n# Plotting streamplot and solver\nfig3, ax = plt.subplots(figsize=(6,6))\nax.streamplot(U, V, du_dxi_streamplot, dv_dxi_streamplot, density = 2)\n# ax.plot(u, v, \'-\', color=\'red\', linewidth=2)\n# ax.plot(u2, v2, \'-\', color=\'red\', linewidth=2)\n# ax.plot(u0_1, v0_1, \'.\', color=\'black\', markersize=15, label=\'\') # Equilibrium/initial condition\n# ax.plo